# Extending the AMM simplified model

In [139]:
import random
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.plotting.backend = "plotly"
import plotly.io as pio
pio.templates.default = "seaborn"
from dataclasses import dataclass
import copy
from dataclasses import field
from typing import List, Callable
from radcad import Model, Simulation, Experiment
from radcad.engine import Engine, Backend

In [140]:
# Run these if you want to play around with the experimentatal section as well 
# For parsing the data from the API
import json
# For downloading data from API
import requests as req

# Extracting onchain data using Subgraph for Uniswap Pool

In [141]:
# Special thanks to CadCAD.edu CadCAD hacks!!!!

# You can explore the subgraph at https://thegraph.com/hosted-service/subgraph/uniswap/uniswap-v2
API_URI = 'https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v2'

# Query for retrieving the history of swaps on a RAI <> WETH Pool
GRAPH_QUERY = '''
{
  swaps (first: 999, where: {pair: "0x8ae720a71622e824f576b4a8c03031066548a3b1"}) {
    sender
    amount0In
    amount1In
    amount0Out
    amount1Out
    timestamp
  }
}
'''

# Retrieve data from query
JSON = {'query': GRAPH_QUERY}
r = req.post(API_URI, json=JSON)
graph_data = json.loads(r.content)['data']

print("Print first 500 characters of the response")
print(r.text[:500])

Print first 500 characters of the response
{"data":{"swaps":[{"sender":"0x0000000089341e263b85d84a0eea39f47c37a9d2","amount0In":"6923.704928904162587556","amount1In":"0","amount0Out":"0","amount1Out":"9.136955397495752446","timestamp":"1622344346"},{"sender":"0xdb7a53e6ae058e1dcf4502341e2adfa522e2b29f","amount0In":"0","amount1In":"2.587173950937311744","amount0Out":"1427.132027126467344622","amount1Out":"0","timestamp":"1662642678"},{"sender":"0x1111111254eeb25477b68fb85ed929f73a960582","amount0In":"0","amount1In":"0.070154574","amount0O


In [142]:
raw_df = pd.DataFrame(graph_data['swaps'])

raw_df.head(5)

,sender,amount0In,amount1In,amount0Out,amount1Out,timestamp
0,0x0000000089341e263b85d84a0eea39f47c37a9d2,6923.704928904162587556,0,0,9.136955397495752446,1622344346
1,0xdb7a53e6ae058e1dcf4502341e2adfa522e2b29f,0,2.587173950937311744,1427.132027126467344622,0,1662642678
2,0x1111111254eeb25477b68fb85ed929f73a960582,0,0.070154574,39.669175274079371826,0,1678140311
3,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,25000,0,0,46.303542416432199552,1615107788
4,0xb2bbfd73971279a457514325f209eb63a0341ef2,1346.92460080000008192,0,0,1.0176696823,1638783764


In [143]:
live_df = (raw_df.assign(timestamp=lambda df: pd.to_datetime(df.timestamp, unit='s'))
            .assign(amount0In=lambda df: pd.to_numeric(df.amount0In))
            .assign(amount1In=lambda df: pd.to_numeric(df.amount1In))
            .assign(amount0Out=lambda df: pd.to_numeric(df.amount0Out))
            .assign(amount1Out=lambda df: pd.to_numeric(df.amount1Out))
            .sort_values('timestamp')
            .reset_index()
            .drop(columns='index')
      )

live_df.head(5)

,sender,amount0In,amount1In,amount0Out,amount1Out,timestamp
0,0x2066c825f210f38bde12e9613399ebc042ac3700,0.0,0.01,0.845403,0.000000,2021-02-13 16:17:00
1,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,40.0,0.00,0.000000,0.415400,2021-02-13 16:32:40
2,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0.0,0.30,92.051817,0.000000,2021-02-14 07:44:08
3,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,100.0,0.00,0.000000,0.224367,2021-02-14 21:41:31
4,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0.0,0.20,99.602708,0.000000,2021-02-16 15:30:45


## Visualizing the trades

In [144]:
live_df['amount1In'].plot()

In [186]:
# create subplot with two plots side by side
from plotly.subplots import make_subplots

fig = make_subplots(rows=2, cols=1)
fig.add_trace(go.Scatter(x=live_df.index, y=live_df['amount0In'].values, name='tokens'), row=1, col=1)
fig.add_trace(go.Scatter(x=live_df.index, y=live_df['amount0Out'].values, name='crypto'), row=2, col=1)

# update subplot layout
fig.update_layout(title='Tokens and coins traded', height = 600, width = 1200)
fig.show()

## Analyzing the Trader Behaviors

In [145]:
live_df.describe()

,amount0In,amount1In,amount0Out,amount1Out
count,999.000000,999.000000,999.000000,999.000000
mean,4426.344081,6.052558,4251.102042,6.176452
std,13077.565203,24.837943,14357.818495,20.222513
min,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.020000,13.478926,0.000000
75%,2284.801044,3.786053,2543.160037,3.430033
max,150000.000000,500.000000,228147.108405,255.585486


In [146]:
unique_wallet_addresses = live_df["sender"].unique()

# Generate a list of IDs starting from 1 to the number of unique wallet addresses
ids = list(range(1, len(unique_wallet_addresses) + 1))

# Create a dictionary that maps the wallet addresses to the corresponding IDs
wallet_to_id = {unique_wallet_addresses[i]: ids[i] for i in range(len(ids))}

# Apply the wallet_to_id dictionary to the wallet address column and store the mapped IDs in the ID column
live_df["ID"] = live_df["sender"].map(wallet_to_id)
live_df = live_df.drop(columns='sender')
live_df.head(10)

,amount0In,amount1In,amount0Out,amount1Out,timestamp,ID
0,0.0,0.010000,0.845403,0.000000,2021-02-13 16:17:00,1
1,40.0,0.000000,0.000000,0.415400,2021-02-13 16:32:40,2
2,0.0,0.300000,92.051817,0.000000,2021-02-14 07:44:08,2
3,100.0,0.000000,0.000000,0.224367,2021-02-14 21:41:31,2
4,0.0,0.200000,99.602708,0.000000,2021-02-16 15:30:45,2
5,0.0,2.500000,1341.728893,0.000000,2021-02-17 13:55:08,2
6,0.0,4.902696,2539.933787,0.000000,2021-02-17 14:07:52,3
7,0.0,35.619156,19114.490270,0.000000,2021-02-17 14:37:22,2
8,0.0,2.731140,1500.000000,0.000000,2021-02-17 14:56:39,2
9,0.0,13.611968,7300.000000,0.000000,2021-02-17 15:20:20,2


In [147]:
live_df['total_token0_traded'] = live_df['amount0In']+ live_df['amount0Out']
summary_df = live_df.groupby('ID').agg({'total_token0_traded': 'sum', 'timestamp':'count'})
summary_df = summary_df.rename(columns={'timestamp':'num_trades'})

In [148]:
summary_df

,total_token0_traded,num_trades
ID,,
1,8.454030e-01,1
2,3.159010e+06,374
3,5.176961e+04,3
4,1.840570e+04,1
5,2.148763e+05,40
...,...,...
135,8.105859e+02,1
136,2.306263e+03,2
137,2.857097e+02,3


In [149]:
summary_df['log_num_trades'] = np.log(summary_df['num_trades'])
summary_df['log_tokens_traded'] = np.log(summary_df['total_token0_traded'])

In [150]:
px.scatter(summary_df, x='num_trades',y='total_token0_traded', height=600, width=1000, log_x=True, log_y=True)

## Clustering Traders into Personas based on trading behavior 

In [151]:
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

X = summary_df[['log_num_trades', 'log_tokens_traded']].values.astype('float64')

# Scale the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Instantiate a DBSCAN object and fit it to the data
dbscan = DBSCAN(eps=0.63, min_samples=5)
# change these hyper parameters to get the clusters that make sense ^ !!!!!!!!!!!!!!

dbscan.fit(X_scaled)

# Get the cluster labels and the number of clusters
labels = dbscan.labels_
n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
print(f"Number of clusters: {n_clusters}")

# Add the cluster labels to the dataframe
summary_df['cluster'] = labels



Number of clusters: 2


In [152]:
# Clustering all the high volume transactions into a new cluster
summary_df.loc[summary_df['num_trades'] > 100, 'cluster'] = n_clusters
print(f"Number of clusters: {n_clusters +1}")

Number of clusters: 3


In [153]:
px.scatter(summary_df, x='num_trades',y='total_token0_traded', color='cluster', height=600, width=1000,log_x=True, log_y=True, template='plotly_white')

In [154]:
clustered = summary_df.groupby('cluster')
mean_total_amt_traded = clustered.mean()['total_token0_traded']
mean_num_trades = clustered.mean()['num_trades']
std_total_amt_traded = clustered.std()['total_token0_traded']
std_num_trades = clustered.std()['num_trades']
persona_df = pd.DataFrame({
    'mean_total_amt_traded':mean_total_amt_traded,
    'std_total_amt_traded':std_total_amt_traded,
    'mean_num_trades':mean_num_trades,
    'std_num_trades':std_num_trades,
})
persona_df

,mean_total_amt_traded,std_total_amt_traded,mean_num_trades,std_num_trades
cluster,,,,
-1,3.062857e+03,4330.337846,9.500,12.020815
0,6.180257e+04,105258.277508,6.875,8.033750
1,9.989789e+03,18820.135956,1.000,0.000000
2,3.159010e+06,NaN,374.000,NaN


Now that we have means and the standard deviations of our trading personas we can use them to build out groups of agents in our future models

## Rounding our data to make more sense with our model 

In [155]:
live_df['rounded_amount0In'] = np.ceil(live_df['amount0In'])
live_df['rounded_amount0Out'] = np.ceil(live_df['amount0Out'])

In [156]:
tradesList = live_df['rounded_amount0In'] - live_df['rounded_amount0Out']

In [157]:
tradesList.head(20)

0        -1.0
1        40.0
2       -93.0
3       100.0
4      -100.0
5     -1342.0
6     -2540.0
7    -19115.0
8     -1500.0
9     -7300.0
10   -15000.0
11     -441.0
12    -5815.0
13     -107.0
14     -673.0
15      118.0
16    -2547.0
17     1573.0
18      752.0
19     -139.0
dtype: float64

# Model

In [158]:
TOKENS = int
PERCENTAGE = float
CENTS = int

In [166]:
#utils
def default(obj):
    return field(default_factory=lambda: copy.copy(obj))


def chooseTrade(Timestep):
    return tradesList[Timestep]

Run =0
Timestep = 333

@dataclass
class Parameters:
    # crash_chance is the chance of crashing in the beginning
    initial_tokens_circulating: PERCENTAGE = default([500000])
    trading_process: List[Callable[[Run, Timestep], int]] = default([chooseTrade])


# Initialize Parameters instance with default values
system_params = Parameters().__dict__

In [167]:

@dataclass
class StateVariables:
    tokens: TOKENS = 500000000000
    crypto: TOKENS = 100000000000000000
    tokens_sold: TOKENS = 0
    tokens_bought: TOKENS = 0
    tokens_in_circulation: TOKENS = 500000
    coin_market_value: CENTS = 159040
    price : CENTS = 31.8

initial_state = StateVariables().__dict__

In [174]:
## Behavior policies

def p_amount_bought_or_sold(params, substep, state_history, prev_state, **kwargs):
    '''policy that decides whether tokens are bought or sold in the block'''
    
    trade=params['trading_process'](prev_state['timestep'])

    if trade>0:
         token_buy = 0
         token_sell = trade
    else:
         token_buy = -trade
         token_sell=0


    # much better ways to do this like just having randint(-1000,1000) and assuming negative = bought, positive = sold
    # but let's stick to doing it the way the machinations model did it
    
    return {'tbought': token_buy, 'tsold': token_sell}


## Mechanism policies 

def p_amount_of_coins(params, substep, state_history, prev_state, **kwargs):
    '''amount of coins to update'''

    k = prev_state['crypto'] * prev_state['tokens']

    if prev_state['tokens_sold'] >0:
        amount_coins_added = 0
        amount_coins_removed=np.round(prev_state['crypto'] - (k/(prev_state['tokens'] + prev_state['tokens_sold'])))

    elif prev_state['tokens_bought'] >0:
        amount_coins_removed = 0
        amount_coins_added = np.round((k/(prev_state['tokens'] - prev_state['tokens_bought'])) - prev_state['crypto'])

    else:
        amount_coins_added =0
        amount_coins_removed=0

    return {'amount_coins_added': amount_coins_added, 'amount_coins_removed': amount_coins_removed}

def p_price(params, substep, state_history, prev_state, **kwargs):
        '''Calculates the price '''
        new_price=(prev_state['crypto']/ prev_state['tokens'])*prev_state['coin_market_value']/1000000000

        return {'price': new_price}


In [175]:
def s_update_tokens_sold(params, substep, state_history, prev_state, policy_input, **kwargs):
        '''Update the tokens sold'''
        updated_tokens_sold = np.round(policy_input['tsold'])
        return ('tokens_sold', max(updated_tokens_sold, 0))

def s_update_tokens_bought(params, substep, state_history, prev_state, policy_input, **kwargs):
        '''Update the state of the difficulty variable by the amount of difficulty increase'''

        updated_tokens_bought = np.round(policy_input['tbought'])
        return ('tokens_bought', max(updated_tokens_bought, 0))


#############################


def s_update_tokens(params, substep, state_history, prev_state, policy_input, **kwargs):
        '''Update the state of the distance variable by the amount of distance sprinted'''
        updated_tokens = np.round(prev_state['tokens'] + prev_state['tokens_sold'] - prev_state['tokens_bought'])

        return ('tokens', max(updated_tokens, 0))


def s_update_crypto(params, substep, state_history, prev_state, policy_input, **kwargs):
        '''Update the state of the coins variable by the amount of new coins generated'''

        updated_coins = np.round(prev_state['crypto'] + policy_input['amount_coins_added'] - policy_input['amount_coins_removed'])

        return ('crypto', max(updated_coins, 0))


def s_update_tokens_in_circulation(params, substep, state_history, prev_state, policy_input, **kwargs):

        '''Update the state of the difficulty variable by the amount of difficulty increase'''

        updated_TIC = np.round(prev_state['tokens_in_circulation']  - prev_state['tokens_sold'] + prev_state['tokens_bought'])

        return ('tokens_in_circulation', max(updated_TIC, 0))



def s_update_coin_market_value(params, substep, state_history, prev_state, policy_input, **kwargs):
        
        '''Update the state of the crash variable'''

        updated_value =159040

        return ('coin_market_value', max(updated_value, 0))



def s_update_price(params, substep, state_history, prev_state, policy_input, **kwargs):
        '''Update the state of the price'''
        
        updated_price = policy_input['price']

        return ('price', max(updated_price, 0))


#####################

state_update_blocks = [
    {
        'policies': {
            'p_amount_bought_or_sold': p_amount_bought_or_sold,
        },

        'variables': {
            'tokens_sold': s_update_tokens_sold,
            'tokens_bought': s_update_tokens_bought
        }
    },
    
    {
        'policies': {
            'p_amount_coins': p_amount_of_coins,
        },

        'variables': {
            'tokens': s_update_tokens,
            'crypto' : s_update_crypto,
            'tokens_in_circulation':s_update_tokens_in_circulation,
            'coin_market_value':s_update_coin_market_value,
        }
    },
    {
        'policies': {
            'p_price': p_price
        },

        'variables': {
            'price': s_update_price
        }
    },

]



In [176]:

# config and run

#number of timesteps
TIMESTEPS = 600
#number of monte carlo runs
RUNS = 1


model = Model(initial_state=initial_state, state_update_blocks=state_update_blocks, params=system_params)
simulation = Simulation(model=model, timesteps=TIMESTEPS, runs=RUNS)

experiment = Experiment(simulation)
# Select the Pathos backend to avoid issues with multiprocessing and Jupyter Notebooks
experiment.engine = Engine(backend=Backend.PATHOS)

result = experiment.run()


df = pd.DataFrame(result)
df

,tokens,crypto,tokens_sold,tokens_bought,tokens_in_circulation,coin_market_value,price,simulation,subset,run,substep,timestep
0,5.000000e+11,1.000000e+17,0.0,0.0,500000.0,159040,31.800000,0,0,1,0,0
1,5.000000e+11,1.000000e+17,0.0,1.0,500000.0,159040,31.800000,0,0,1,1,1
2,5.000000e+11,1.000000e+17,0.0,1.0,500001.0,159040,31.800000,0,0,1,2,1
3,5.000000e+11,1.000000e+17,0.0,1.0,500001.0,159040,31.808000,0,0,1,3,1
4,5.000000e+11,1.000000e+17,40.0,0.0,500001.0,159040,31.808000,0,0,1,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...
1796,5.000003e+11,9.999995e+16,14896.0,0.0,237837.0,159040,31.807969,0,0,1,2,599
1797,5.000003e+11,9.999995e+16,14896.0,0.0,237837.0,159040,31.807967,0,0,1,3,599
1798,5.000003e+11,9.999995e+16,10613.0,0.0,237837.0,159040,31.807967,0,0,1,1,600
1799,5.000003e+11,9.999995e+16,10613.0,0.0,227224.0,159040,31.807967,0,0,1,2,600


In [177]:
df2 = df[df['substep']==3].reset_index()

In [181]:
df2['tokens_bought'].plot()

In [189]:
df2['crypto'].plot(width = 1300)